In [ ]:
# Weather Helper ReAct Agent

!pip install httpx


In [ ]:

import os
import httpx
import re

DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")

DEEPSEEK_API_URL = "https://api.deepseek.com/v1/chat/completions"

def deepseek_chat_completion(messages, model="deepseek-chat", temperature=0):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }
    response = httpx.post(DEEPSEEK_API_URL, headers=headers, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        return deepseek_chat_completion(self.messages)

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

current_weather:
e.g. current_weather: Barcelona
Returns the current weather for the given city.

forecast:
e.g. forecast: Barcelona
Returns the weather forecast for the given city.

Example session:

Question: What is the weather like in Barcelona?
Thought: I should get the current weather for Barcelona.
Action: current_weather: Barcelona
PAUSE

You will be called again with this:

Observation: The current weather in Barcelona is 22°C, sunny.

You then output:

Answer: The current weather in Barcelona is 22°C, sunny.
""".strip()

def current_weather(city):
    # Mock data for demonstration
    weather_data = {
        "barcelona": "The current weather in Barcelona is 22°C, sunny.",
        "madrid": "The current weather in Madrid is 25°C, partly cloudy.",
        "london": "The current weather in London is 16°C, rainy.",
        "paris": "The current weather in Paris is 18°C, cloudy."
    }
    return weather_data.get(city.lower(), f"Sorry, I don't have weather data for {city}.")

def forecast(city):
    # Mock data for demonstration
    forecast_data = {
        "barcelona": "The forecast for Barcelona is 24°C and sunny tomorrow.",
        "madrid": "The forecast for Madrid is 27°C and sunny tomorrow.",
        "london": "The forecast for London is 15°C and rain tomorrow.",
        "paris": "The forecast for Paris is 19°C and partly cloudy tomorrow."
    }
    return forecast_data.get(city.lower(), f"Sorry, I don't have forecast data for {city}.")

known_actions = {
    "current_weather": current_weather,
    "forecast": forecast
}

action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running", action, action_input)
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return

In [2]:

# Example usage:
query("What is the weather like in Barcelona?")


Thought: I should get the current weather for Barcelona.
Action: current_weather: Barcelona
PAUSE
 -- running current_weather Barcelona
Observation: The current weather in Barcelona is 22°C, sunny.
Answer: The current weather in Barcelona is 22°C, sunny.


In [3]:
query("What is the forecast for London?")

Thought: I should get the weather forecast for London.
Action: forecast: London
PAUSE
 -- running forecast London
Observation: The forecast for London is 15°C and rain tomorrow.
Answer: The forecast for London is 15°C and rain tomorrow.
